# Specifying priors

This tutorial demonstrates how to specify parameter priors, including:

* uniform and log-uniform distributions
* gaussian and more complicated distributions
* multi-dimensional priors (not factorized)
* non-analytic priors
* priors on fractions

In [1]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib notebook

## Cumulative prior distributions

Any 1-dimensional probability distribution is normalised so that its integral is 1. That is, the cumulative distribution goes from 0 to 1. For example:

In [2]:
gaussdistribution = scipy.stats.norm(2, 0.3)
uniformdistribution = scipy.stats.uniform(3.5, 1.2)
x = np.linspace(0, 5, 400)
plt.figure()
plt.plot(x, gaussdistribution.pdf(x), '--', label='density (Gauss)')
plt.plot(x, gaussdistribution.cdf(x), label='cumulative (Gauss)')
plt.plot(x, uniformdistribution.pdf(x), '--', label='density (uniform)')
plt.plot(x, uniformdistribution.cdf(x), label='cumulative (uniform)')
plt.ylabel('Probability')
plt.xlabel('Model parameter x')
plt.legend();

<IPython.core.display.Javascript object>

## Transforming from the unit interval

We invert the cumulative probability distribution mapping quantiles (0...1) to the corresponding model parameter value.

Lets start with the uniform distribution.

In [3]:
def transform_1d_uniform(quantile):
    lower_bound = 3.5
    width = 1.2
    return quantile * width + lower_bound
    

Scipy provides the inverse cumulative probability distributions:

In [4]:
def transform_1d(quantile):
    return gaussdistribution.ppf(quantile)


UltraNest samples from the unit interval to obtain prior samples. Lets try drawing a few examples from our function:

In [5]:
uniform_samples = transform_1d_uniform(np.random.uniform(0, 1, size=100000))
gauss_samples = transform_1d(np.random.uniform(0, 1, size=100000))
plt.figure()
plt.hist(uniform_samples, bins=20, histtype='step', density=True, label='Uniform')
plt.hist(gauss_samples, bins=100, histtype='step', density=True, label='Gauss')
plt.xlabel('Model parameter x')
plt.ylabel('Density')
plt.legend();

<IPython.core.display.Javascript object>

Beautiful! We obtained nice samples that follow the prior distribution.

## The unit hypercube

Lets specify a prior for UltraNest with multiple parameters:

* a uniform distribution from 3.5 to 4.7
* a log-uniform distribution from 0.01 to 100
* a gaussian distribution around 2.0 +- 0.3


In [6]:
# out transform function will receive one quantile corresponding to each of the three parameter
def transform(quantile_cube):
    # prepare the output array, which has the same shape
    transformed_parameters = np.empty_like(quantile_cube)
    # first parameter: a uniform distribution
    transformed_parameters[0] = 3.5 + 1.2 * quantile_cube[0]
    # second parameter: a log-uniform distribution
    transformed_parameters[1] = 10**(-2 + 4 * quantile_cube[1])
    # third parameter: Gaussian
    transformed_parameters[2] = mydistribution.ppf(quantile_cube[2])
    
    return transformed_parameters

Some recommendations:

* [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions) provides many 1-d distributions that can be used like this.
* avoid building scipy.stats objects in the transform, because this is slow -- build them outside first, then only invoke the .ppf method in the transform.


## Dependent priors

In some cases, a previous experiment gives informative priors which we want to incorporate, and they may be inter-dependent. For example, consider a two-dimensional gaussian prior distribution:


In [7]:
means = np.array([2, 3])
cov = np.array([[1, 0.6], [0.6, 0.4]])

In [8]:
rv = scipy.stats.multivariate_normal(means, cov)
x, y = np.linspace(-1, 5, 400), np.linspace(1.5, 5, 400)
X, Y = np.meshgrid(x, y)
Z = rv.pdf(np.transpose([X.flatten(), Y.flatten()])).reshape(X.shape)
plt.figure()
plt.title('Correlated prior')
plt.contourf(X, Y, Z, cmap='magma_r')
plt.xlabel('Parameter 1')
plt.ylabel('Parameter 2');

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Parameter 2')

We recall:

* Parameter 1 has a cumulative distribution
* At each value of Parameter 1, Parameter 2 has a cumulative distribution.
* We can thus specify a dependent distribution using the unit hypercube

In [9]:
a = np.linalg.inv(cov)
l, v = np.linalg.eigh(a)
rotation_matrix = np.dot(v, np.diag(1. / np.sqrt(l)))

def transform_correlated(quantiles):
    # sample a independent multivariate gaussian
    independent_gaussian = scipy.stats.norm.ppf(quantiles)
    # rotate and shift
    return means + np.einsum('ij,kj->ki', rotation_matrix, independent_gaussian)

Lets try sampling!

In [10]:
samples = transform_correlated(np.random.uniform(0, 1, size=(100, 2)))

plt.figure()
plt.title('Correlated prior')
plt.contourf(X, Y, Z, cmap='magma_r')
plt.plot(samples[:,0], samples[:,1], 'o', mew=1, mfc='w', mec='k')
plt.xlabel('Parameter 1')
plt.ylabel('Parameter 2');


<IPython.core.display.Javascript object>

Text(0, 0.5, 'Parameter 2')

A similar effect can be achieved by defining transforms in sequence (this is a different prior though):

In [11]:
gauss1 = scipy.stats.norm(2, 1)
gauss2 = scipy.stats.norm(0, 0.1)


def transform_correlated(quantiles):
    parameters = np.empty_like(quantiles)
    # first parameter is independent
    parameters[0] = gauss1.ppf(quantiles[0])
    # second parameter depends on first parameter, here with a shift
    parameters[1] = parameters[0] + gauss2.ppf(quantiles[0])
    return parameters

## Non-analytic priors

Sometimes, the prior may not be easily invertable. For example, when it is given as posterior samples from a previous analysis. I



In [12]:
posterior_samples = np.hstack((np.random.uniform(0, 3, 2000), np.random.normal(3, 0.2, 2000)))

plt.figure(figsize=(4,2))
plt.hist(posterior_samples, histtype='step', bins=100);


<IPython.core.display.Javascript object>

(array([ 22.,  18.,  32.,  18.,  27.,  26.,  42.,  34.,  19.,  25.,  22.,
         33.,  26.,  29.,  18.,  31.,  18.,  13.,  22.,  27.,  21.,  34.,
         25.,  21.,  27.,  20.,  36.,  32.,  20.,  32.,  36.,  20.,  25.,
         27.,  23.,  29.,  22.,  18.,  29.,  31.,  32.,  15.,  25.,  20.,
         25.,  24.,  25.,  23.,  29.,  32.,  35.,  27.,  31.,  40.,  30.,
         30.,  29.,  18.,  23.,  25.,  32.,  24.,  18.,  27.,  31.,  52.,
         55.,  53.,  76.,  70., 102., 137., 128., 161., 167., 180., 165.,
        177., 150., 116., 105., 108.,  90.,  66.,  45.,  31.,  25.,  12.,
         10.,   9.,   6.,   2.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,
          1.]),
 array([0.00642089, 0.04542824, 0.08443558, 0.12344292, 0.16245026,
        0.2014576 , 0.24046495, 0.27947229, 0.31847963, 0.35748697,
        0.39649431, 0.43550166, 0.474509  , 0.51351634, 0.55252368,
        0.59153102, 0.63053837, 0.66954571, 0.70855305, 0.74756039,
        0.78656773, 0.82557508, 0.86458242, 0.

In this case, you can compute the cumulative distribution numerically and invert it:

In [13]:
hist, bin_edges = np.histogram(posterior_samples, bins=100)
hist_cumulative = np.cumsum(hist / hist.sum())
bin_middle = (bin_edges[:-1] + bin_edges[1:]) / 2

def transform_histogram(quantile):
    return np.interp(quantile, hist_cumulative, bin_middle)

samples = transform_histogram(np.random.uniform(size=1000))
plt.figure(figsize=(4,2))
plt.hist(posterior_samples, histtype='step', bins=100, density=True);
plt.hist(samples, histtype='step', bins=100, density=True);


<IPython.core.display.Javascript object>

(array([0.14493799, 0.26088839, 0.0579752 , 0.31886359, 0.14493799,
        0.23190079, 0.26088839, 0.37683879, 0.0579752 , 0.14493799,
        0.1159504 , 0.34785119, 0.0579752 , 0.0289876 , 0.20291319,
        0.0289876 , 0.26088839, 0.1159504 , 0.0289876 , 0.17392559,
        0.14493799, 0.26088839, 0.1159504 , 0.28987599, 0.20291319,
        0.0579752 , 0.20291319, 0.26088839, 0.20291319, 0.26088839,
        0.14493799, 0.28987599, 0.1159504 , 0.14493799, 0.0869628 ,
        0.14493799, 0.17392559, 0.14493799, 0.31886359, 0.28987599,
        0.1159504 , 0.1159504 , 0.17392559, 0.14493799, 0.1159504 ,
        0.        , 0.20291319, 0.0869628 , 0.14493799, 0.26088839,
        0.26088839, 0.1159504 , 0.0869628 , 0.0869628 , 0.14493799,
        0.23190079, 0.23190079, 0.17392559, 0.17392559, 0.31886359,
        0.23190079, 0.34785119, 0.23190079, 0.1159504 , 0.20291319,
        0.14493799, 0.0869628 , 0.20291319, 0.14493799, 0.0869628 ,
        0.17392559, 0.0579752 , 0.34785119, 0.40

## Fraction parameters

Some parameters such as fractions (or abundances) may be required to sum to 1. How can we specify such parameters?

One option is to use absolute numbers. For example, instead of specifying the total mass and mass fractions for each chemical element, parameterise the mass of each element. This avoids the <=1 constraint, and may be easier to infer. A drawback is that the prior ranges for each element mass may be wide.

The other option is to use the right distribution exactly made for this, which samples unbiased under the constraint (sum<=1): The [Dirichlet distribution](https://en.wikipedia.org/wiki/Dirichlet_distribution). Here we assume that the prior on the individual fraction is flat (flat Dirichlet distribution, $\alpha=1$).


In [14]:
def transform_dirichlet(quantiles):
    # https://en.wikipedia.org/wiki/Dirichlet_distribution#Random_number_generation
    # first inverse transform sample from Gamma(alpha=1,beta=1), which is Exponential(1)
    gamma_quantiles = -np.log(quantiles)
    # dirichlet variables
    return gamma_quantiles / gamma_quantiles.sum(axis=1).reshape((-1, 1))


Lets have a look at the samples:

In [15]:
samples = transform_dirichlet(np.random.uniform(0, 1, size=(400, 3)))

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=10., azim=-30)
ax.plot(samples[:,0], samples[:,1], samples[:,2], 'x ');

<IPython.core.display.Javascript object>

The samples nicely lie on the plane where the sum is 1.

##  Further topics

Check out the rest of the documentation and the tutorials.

They illustrate how to:

* [speeding up transforms with vectorisation](https://johannesbuchner.github.io/UltraNest/performance.html)
* [How to use priors in Ultranest](https://johannesbuchner.github.io/UltraNest/usage-spectral-line.html)
* and many example tutorials